In [3]:
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import geopandas as gpd
import pandas as pd
import os
import logging 
from shapely import geometry
import requests
import re
from bs4 import BeautifulSoup
import glob
from zipfile import ZipFile
import shutil
import urllib
import json

NumExpr defaulting to 8 threads.


In [4]:
# get top-level logger object
logger = logging.getLogger()
for handler in logger.handlers: logger.removeHandler(handler)
# manually set level 
logger.setLevel(logging.INFO)
# print to console
console = logging.StreamHandler()
logger.addHandler(console)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [5]:
data_dir = 'data'

In [12]:
CARTO_USER = os.getenv('CARTO_WRI_RW_USER')
CARTO_KEY = os.getenv('CARTO_WRI_RW_KEY')
set_default_credentials(username=CARTO_USER, base_url="https://{user}.carto.com/".format(user=CARTO_USER),api_key=CARTO_KEY)

In [8]:
# insert the url used to download the data from the source website

ids = ["133555","133497","133513","133543","133501","133502"," 133409","133448","133492"]

url_template= 'https://www.climatewatchdata.org/api/v1/data/ndc_content?indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&per_page=1000'
url = url_template.format(*tuple(ids)) + '&page={}'
print(url)


https://www.climatewatchdata.org/api/v1/data/ndc_content?indicator_ids[]=133555&indicator_ids[]=133497&indicator_ids[]=133513&indicator_ids[]=133543&indicator_ids[]=133501&indicator_ids[]=133502&indicator_ids[]= 133409&indicator_ids[]=133448&indicator_ids[]=133492&per_page=1000&page={}


In [9]:
paginated_dfs = []
# download the cata from the source
for i in range(1,2):
    req = requests.get(url)
    text_data= req.text
    json_dict= json.loads(text_data)
    df = pd.DataFrame.from_dict(json_dict['data'])
    paginated_dfs.append(df)

df = pd.concat(paginated_dfs)


In [10]:
print(df.head())

         id source iso_code3      country global_category overview_category  \
0  28340163     WB       AFG  Afghanistan                                     
1  28348831     WB       AFG  Afghanistan                                     
2  28349816     WB       AFG  Afghanistan                                     
3  28350604     WB       AFG  Afghanistan                                     
4  28350801     WB       AFG  Afghanistan                                     

         sector                  subsector                      indicator_id  \
0        Energy    Renewable Energy: Ocean     m_renewable_energy_ocean_auto   
1     Transport                   Maritime                   m_maritime_auto   
2  Coastal Zone          Coastal fisheries          a_coastal_fisheries_auto   
3  Coastal Zone  Sea-level Rise Protection  a_sea_level_rise_protection_auto   
4   Agriculture  Fisheries and Aquaculture  a_fisheries_and_aquaculture_auto   

              indicator_name                

In [11]:
df['count'] = 1
df_world = df.groupby(['indicator_id']).agg({'indicator_name':'first', 'sector':'first', 'subsector':'first','count':'sum', 'value':'first'}).reset_index()
df_world['iso_code3'] = 'GLB'
df_world['country'] = 'Global'
df = pd.concat([df, df_world])

In [13]:
logger.info('Upload dataframe to Carto')
# upload data frame to Carto
to_carto(df, 'ocn_025_rw0_ocean_ndc_measures', if_exists='replace')

Upload dataframe to Carto


CartoException: ['invalid input syntax for type double precision: "0    28340163.0"']